In [6]:
from tensorflow.keras import backend as K
from tensorflow.keras.applications import inception_v3

In [8]:
K.set_learning_phase(0)

In [9]:
model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)

87916544/87910968 [==============================] - 186s 2us/step


In [11]:
layer_contriburions = {
    'mixed2': 0.2,
    'mixed3': 3.0,
    'mixed4': 2.0,
    'mixed5':1.5,
}

In [14]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [15]:
layer_dict

{'input_1': <tensorflow.python.keras.engine.input_layer.InputLayer at 0x7ff5b1eb9fd0>,
 'conv2d': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7ff5b1f5d310>,
 'batch_normalization': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x7ff5b2292150>,
 'activation': <tensorflow.python.keras.layers.core.Activation at 0x7ff5b1e51950>,
 'conv2d_1': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7ff5b1ec0150>,
 'batch_normalization_1': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x7ff5b05f7bd0>,
 'activation_1': <tensorflow.python.keras.layers.core.Activation at 0x7ff5b1ee3650>,
 'conv2d_2': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7ff5b1f32a10>,
 'batch_normalization_2': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x7ff5f862e790>,
 'activation_2': <tensorflow.python.keras.layers.core.Activation at 0x7ff5b1f6e0d0>,
 'max_pooling2d': <tensorflow.python.keras.layers.pooling.MaxPooling2D at